# Definition

In [1]:
def readdata_and_savemodel(filename):
    #INPUT = 1000001000100010001.csv,1000001000100010002.csv,1000001000100010003.csv,1000001000100010004.csv,1000001000100010011.csv
    #OUTPUT : 모델이 경로에 저장됨
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.feature_extraction.text import TfidfTransformer
    from keras.utils import to_categorical
    from sklearn.preprocessing import LabelEncoder,OneHotEncoder
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.callbacks import ModelCheckpoint
    import os
    
    # 파일로부터 X,Y데이터를 읽어서 전처리(백터화 및 원핫인코딩)
    x_data,y_data = load_from_dataset(filename) #읽어옴
    x_train = make_x_train(x_data) #전처리
    y_train = make_y_train(y_data) #전처리
    name=filename.split(".")[0] #파일이름에서 카테고리 키워드를 받아옴

    #모델 저장경로 설정
    MODEL_DIR = './model/'
    if not os.path.exists(MODEL_DIR):
        os.mkdir(MODEL_DIR)
    modelpath = './model/'+name+'.hdf5'
    checkpointer = ModelCheckpoint(filepath=modelpath, monitor = 'loss', save_best_only=True, verbose=1)

    #모델 선언 및 layer설정
    model = Sequential()
    model.add(Dense(64,input_dim = len(x_train[0]),activation='relu'))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(len(y_train[0]),activation='softmax'))
    model.compile(loss = 'categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    #가장 마지막에 있는 모델 하나만 저장
    history = model.fit(x_train,y_train,epochs=2000,verbose=1,batch_size=1,callbacks=[checkpointer])

    
def make_y_train(result2):
    #INPUT : load_from_dataset로 읽어온 Y값 전처리 함수
    import numpy as np
    from keras.utils import to_categorical
    from sklearn.preprocessing import OneHotEncoder,LabelEncoder
    f=LabelEncoder()
    f.fit(result2)
    y_train = f.transform(result2)
    #OUTPUT = [0....1....0] 원핫벡터
    return to_categorical(y_train)

def x_onehot_encoding(input_list):
    # X데이터 Preprocessing 하는 1단계함수 직접사용할 일이 거의 없음
    # make_x_train 함수에서 사용됨
    # INPUT = ['복합성', '웜톤', '트러블', '모공', '민감성', '잡티']
    name_list=["복합성","건성","지성","쿨톤","웜톤","잡티","미백","주름","각질","트러블","블랙헤드","피지과다","민감성","모공","탄력","홍조","아토피"]
    result=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    for i in input_list:
        index = name_list.index(i)
        result[index]+=1
    # OUTPUT = [1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0]
    return result

def make_x_train(result1):
    #INPUT : load_from_dataset로 읽어온 Y값 전처리 함수
    # X데이터를 가공하는함수
    # 내부에서 x_onehot_encoding을 사용하며 for문으로 모든 x값을 처리
    import numpy as np
    temp=[]
    result = list(result1)
    for i in result:
        temp.append(x_onehot_encoding(i))
    #여기서 나온 값을 바로 model.fit 시키면 됨
    #OUTPUT
    #[
    #    [1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0],
    #    [1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0],
    #    [1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0]
    #]
    return np.array(temp)

def load_from_dataset(string):
    # INPUT : string은 csv 파일 이름
    import pandas as pd
    df = pd.read_csv(string,encoding='utf-8')
    y_data = df.values[:,1]
    x_data = df.values[:,-1]
    temp=[]
    for i in x_data:
        willappend = i.split(" ")
        willappend.pop()
        temp.append(willappend)
    # csv파일로 부터 x값, y값을 각각 읽어서 반환
    # 아직 전처리 안됨
    return temp,y_data

def load_model_hdf5(filename):
    # INPUT : 1000001000100010001.csv
    # hdf파일 읽어옴
    from tensorflow import keras
    loaded_model = keras.models.load_model('model/'+filename)
    # load한 모델을 리턴
    return loaded_model

def predict_code_value(category_name,input_value):
    # categoty_name  ex)로션, 세럼, 토너
    #input_value = ["복합성",등등등]
    categoryno = Name_to_CategoryNo(category_name)
    import numpy as np
    import pandas as pd
    b=np.array(x_onehot_encoding(input_value)).reshape(1,-1)
    model = load_model_hdf5(categoryno+'.hdf5')
    x,y=load_from_dataset(categoryno+'.csv')
    df = pd.read_csv(categoryno+'.csv',encoding='utf-8')
    code = y[model.predict_classes(b)]
    url = 'https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo={}&dispCatNo={}'.format(code[0],categoryno)
    #          'A000000103112'
    # 이름 가격 url
    row = df[df['id']==code[0]]
    pre_price = row['price'].astype(str)
    pre_name = row['name'].astype(str)
    price=text_processing(pre_price.iloc[0])
    name=text_processing(pre_name.iloc[0])
    return name,price,url 

def get_url(code,filename):
    #url을 가공해서 리턴하는 함수
    catno = filename.split(".")[0]
    base_url='https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?'+'goodsNo={}&dispCatNo={}'.format(code,catno)
    # 필요에 따라 print함수와 함께 사용
    return base_url

def GoodsNo_to_Name(goodsnum):
    # 상품번호에서 이름으로 변환하는 함수
    if goodsnum=='1000001000100010001' or goodsnum==1000001000100010001:
        return "스킨/토너"
    elif goodsnum=='1000001000100010002' or goodsnum==1000001000100010002:
        return "로션"
    elif goodsnum=='1000001000100010003' or goodsnum==1000001000100010003:
        return "에센스/세럼"
    elif goodsnum=='1000001000100010011' or goodsnum==1000001000100010011:
        return "앰플"
    elif goodsnum=='1000001000100010004' or goodsnum==1000001000100010004:
        return "크림"
    else:
        print("없는 코드입니다.")
        return 

def Name_to_CategoryNo(category_name):
    # 이름에서 상품번호로 변환하는 함수
    if category_name=='토너' or category_name=='스킨' or category_name=='스킨토너' or category_name=='토너스킨' or category_name=="스킨/토너" or category_name=="토너/스킨":
        return '1000001000100010001'
    elif category_name=='로션':
        return '1000001000100010002'
    elif category_name=='에센스' or category_name=='세럼' or category_name=='새럼' or category_name=='에센스/세럼' or category_name=='에센스/새럼' or category_name=='세럼/에센스' or category_name=='새럼/에센스':
        return '1000001000100010003'
    elif category_name=='앰플' or category_name=='엠플':
        return '1000001000100010011'
    elif category_name=='크림':
        return '1000001000100010001'
    else:
        print("없는 코드코드입니다.")
        return 
def get_url(goods_num,filename):
    #url을 가공해서 리턴하는 함수
    catno = filename.split(".")[0]
    base_url='https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?'+'goodsNo={}&dispCatNo={}'.format(goods_num,catno)
    # 필요에 따라 print함수와 함께 사용
    return base_url

def text_processing(string):
    # 택스트 가공함수
    # INPUT string="한번 써보고 너무 좋아서(^_^)매번 구입해서 사용 중이에요:). 바이오더마 제품이라 믿음도 가요.물스킨 타입이라 닦아내는 용도로 쓰고 있는 데 적당히 쿨링감 있고 좋아요저는 건성이긴 하지만 이 제품은 가볍고 깨끗한 느낌이라지성한테 더 잘 어울리는 제품인 거 같아요보습이 강하지는 않고 진정이랑 산뜻함?? 이런 느낌이 강해요. 향도 쎄지 않은 그냥 쿨한 느낌이고 어름에 쓰기 딱 좋은 거 같아요냉장고에 보관하고 사용 중인데 더 피부진정에도움이 되는 거 같아요.자극도 없이 순하고 세네통 넘게 사용 중이에요 남자친구도 안끈적거리고 시원하고 좋다고 애용중입니다:)"
    # 줄바꿈, 특수기호, 불규칙한 인덴트 제거
    file = open("test.txt",'w',encoding='utf-8')
    file.write(string)
    file.write("\n")
    file.close()
    file=open("test.txt",'r',encoding='utf-8')
    result=''
    while True:
        line = file.readline()
        if not line:
            break
        result+=line.replace("\n",'')
    file.close()
    return result

def category_tag_to_dictionary(category_name,input_value):
    a,b,c = predict_code_value(category_name,input_value)
    result = {
        'name' : a,
        "price" : b,
        "url" : c,
    }
    return result

#readdata_and_savemodel("1000001000100010001.csv")

### 모델 저장

In [8]:
readdata_and_savemodel("1000001000100010001.csv")

Epoch 1/2000
240/240 [==============================] - 0s 2ms/step - loss: 5.6000 - accuracy: 0.0000e+00

Epoch 00001: loss improved from inf to 5.60004, saving model to ./model/1000001000100010001.hdf5
Epoch 2/2000
240/240 [==============================] - 0s 802us/step - loss: 5.2889 - accuracy: 0.0250

Epoch 00002: loss improved from 5.60004 to 5.28885, saving model to ./model/1000001000100010001.hdf5
Epoch 3/2000
240/240 [==============================] - 0s 792us/step - loss: 4.3112 - accuracy: 0.0917

Epoch 00003: loss improved from 5.28885 to 4.31118, saving model to ./model/1000001000100010001.hdf5
Epoch 4/2000
240/240 [==============================] - 0s 789us/step - loss: 3.2111 - accuracy: 0.2833

Epoch 00004: loss improved from 4.31118 to 3.21114, saving model to ./model/1000001000100010001.hdf5
Epoch 5/2000
240/240 [==============================] - 0s 780us/step - loss: 2.5820 - accuracy: 0.3750

Epoch 00005: loss improved from 3.21114 to 2.58197, saving model to ./mod

Epoch 41/2000
240/240 [==============================] - 0s 736us/step - loss: 1.7783 - accuracy: 0.4083

Epoch 00041: loss did not improve from 1.77800
Epoch 42/2000
240/240 [==============================] - 0s 773us/step - loss: 1.7728 - accuracy: 0.4417

Epoch 00042: loss improved from 1.77800 to 1.77282, saving model to ./model/1000001000100010001.hdf5
Epoch 43/2000
240/240 [==============================] - 0s 756us/step - loss: 1.7719 - accuracy: 0.4167

Epoch 00043: loss improved from 1.77282 to 1.77190, saving model to ./model/1000001000100010001.hdf5
Epoch 44/2000
240/240 [==============================] - 0s 689us/step - loss: 1.7690 - accuracy: 0.4208

Epoch 00044: loss improved from 1.77190 to 1.76897, saving model to ./model/1000001000100010001.hdf5
Epoch 45/2000
240/240 [==============================] - 0s 707us/step - loss: 1.7680 - accuracy: 0.4167TA: 0s - loss: 1.7024 - accuracy: 0.44

Epoch 00045: loss improved from 1.76897 to 1.76797, saving model to ./model/100000

240/240 [==============================] - 0s 759us/step - loss: 1.6903 - accuracy: 0.3958

Epoch 00131: loss improved from 1.69051 to 1.69030, saving model to ./model/1000001000100010001.hdf5
Epoch 132/2000
240/240 [==============================] - 0s 695us/step - loss: 1.6897 - accuracy: 0.4167

Epoch 00132: loss improved from 1.69030 to 1.68966, saving model to ./model/1000001000100010001.hdf5
Epoch 133/2000
240/240 [==============================] - 0s 695us/step - loss: 1.6901 - accuracy: 0.4125

Epoch 00133: loss did not improve from 1.68966
Epoch 134/2000
240/240 [==============================] - 0s 683us/step - loss: 1.6893 - accuracy: 0.4167

Epoch 00134: loss improved from 1.68966 to 1.68929, saving model to ./model/1000001000100010001.hdf5
Epoch 135/2000
240/240 [==============================] - 0s 698us/step - loss: 1.6885 - accuracy: 0.4250

Epoch 00135: loss improved from 1.68929 to 1.68851, saving model to ./model/1000001000100010001.hdf5
Epoch 136/2000
240/240 [=====

240/240 [==============================] - 0s 726us/step - loss: 1.6807 - accuracy: 0.4250

Epoch 00178: loss did not improve from 1.67981
Epoch 179/2000
240/240 [==============================] - 0s 719us/step - loss: 1.6925 - accuracy: 0.4250

Epoch 00179: loss did not improve from 1.67981
Epoch 180/2000
240/240 [==============================] - 0s 729us/step - loss: 1.6824 - accuracy: 0.4333

Epoch 00180: loss did not improve from 1.67981
Epoch 181/2000
240/240 [==============================] - 0s 718us/step - loss: 1.6814 - accuracy: 0.4125

Epoch 00181: loss did not improve from 1.67981
Epoch 182/2000
240/240 [==============================] - 0s 729us/step - loss: 1.6817 - accuracy: 0.4292

Epoch 00182: loss did not improve from 1.67981
Epoch 183/2000
240/240 [==============================] - 0s 712us/step - loss: 1.6844 - accuracy: 0.4083

Epoch 00183: loss did not improve from 1.67981
Epoch 184/2000
240/240 [==============================] - 0s 654us/step - loss: 1.6815 - ac

240/240 [==============================] - 0s 725us/step - loss: 1.6754 - accuracy: 0.4125

Epoch 00227: loss did not improve from 1.67400
Epoch 228/2000
240/240 [==============================] - 0s 717us/step - loss: 1.6737 - accuracy: 0.4208

Epoch 00228: loss improved from 1.67400 to 1.67366, saving model to ./model/1000001000100010001.hdf5
Epoch 229/2000
240/240 [==============================] - 0s 733us/step - loss: 1.6740 - accuracy: 0.4250

Epoch 00229: loss did not improve from 1.67366
Epoch 230/2000
240/240 [==============================] - 0s 713us/step - loss: 1.6743 - accuracy: 0.4208

Epoch 00230: loss did not improve from 1.67366
Epoch 231/2000
240/240 [==============================] - 0s 730us/step - loss: 1.6743 - accuracy: 0.4292

Epoch 00231: loss did not improve from 1.67366
Epoch 232/2000
240/240 [==============================] - 0s 717us/step - loss: 1.6759 - accuracy: 0.4250

Epoch 00232: loss did not improve from 1.67366
Epoch 233/2000
240/240 [=============

...

240/240 [==============================] - 0s 634us/step - loss: 1.6594 - accuracy: 0.4250

Epoch 01568: loss did not improve from 1.65907
Epoch 1569/2000
240/240 [==============================] - 0s 624us/step - loss: 1.6594 - accuracy: 0.4167

Epoch 01569: loss did not improve from 1.65907
Epoch 1570/2000
240/240 [==============================] - 0s 642us/step - loss: 1.6594 - accuracy: 0.4250

Epoch 01570: loss did not improve from 1.65907
Epoch 1571/2000
240/240 [==============================] - 0s 684us/step - loss: 1.6592 - accuracy: 0.4250

Epoch 01571: loss did not improve from 1.65907
Epoch 1572/2000
240/240 [==============================] - 0s 690us/step - loss: 1.6592 - accuracy: 0.4250

Epoch 01572: loss did not improve from 1.65907
Epoch 1573/2000
240/240 [==============================] - 0s 688us/step - loss: 1.6592 - accuracy: 0.4208

Epoch 01573: loss did not improve from 1.65907
Epoch 1574/2000
240/240 [==============================] - 0s 742us/step - loss: 1.659

240/240 [==============================] - 0s 706us/step - loss: 1.6596 - accuracy: 0.4333

Epoch 01621: loss did not improve from 1.65907
Epoch 1622/2000
240/240 [==============================] - 0s 690us/step - loss: 1.6596 - accuracy: 0.4167

Epoch 01622: loss did not improve from 1.65907
Epoch 1623/2000
240/240 [==============================] - 0s 687us/step - loss: 1.6595 - accuracy: 0.4125

Epoch 01623: loss did not improve from 1.65907
Epoch 1624/2000
240/240 [==============================] - 0s 686us/step - loss: 1.6594 - accuracy: 0.4333

Epoch 01624: loss did not improve from 1.65907
Epoch 1625/2000
240/240 [==============================] - 0s 680us/step - loss: 1.6594 - accuracy: 0.4292

Epoch 01625: loss did not improve from 1.65907
Epoch 1626/2000
240/240 [==============================] - 0s 677us/step - loss: 1.6592 - accuracy: 0.4167

Epoch 01626: loss did not improve from 1.65907
Epoch 1627/2000
240/240 [==============================] - 0s 685us/step - loss: 1.659

240/240 [==============================] - 0s 691us/step - loss: 1.6602 - accuracy: 0.4208

Epoch 01725: loss did not improve from 1.65905
Epoch 1726/2000
240/240 [==============================] - 0s 757us/step - loss: 1.6601 - accuracy: 0.4208

Epoch 01726: loss did not improve from 1.65905
Epoch 1727/2000
240/240 [==============================] - 0s 694us/step - loss: 1.6599 - accuracy: 0.4208

Epoch 01727: loss did not improve from 1.65905
Epoch 1728/2000
240/240 [==============================] - 0s 679us/step - loss: 1.6597 - accuracy: 0.4083

Epoch 01728: loss did not improve from 1.65905
Epoch 1729/2000
240/240 [==============================] - 0s 683us/step - loss: 1.6596 - accuracy: 0.4292

Epoch 01729: loss did not improve from 1.65905
Epoch 1730/2000
240/240 [==============================] - 0s 811us/step - loss: 1.6594 - accuracy: 0.4083

Epoch 01730: loss did not improve from 1.65905
Epoch 1731/2000
240/240 [==============================] - 0s 789us/step - loss: 1.659

240/240 [==============================] - 0s 622us/step - loss: 1.6590 - accuracy: 0.4292

Epoch 01829: loss did not improve from 1.65897
Epoch 1830/2000
240/240 [==============================] - 0s 617us/step - loss: 1.6591 - accuracy: 0.4250

Epoch 01830: loss did not improve from 1.65897
Epoch 1831/2000
240/240 [==============================] - 0s 613us/step - loss: 1.6591 - accuracy: 0.4042

Epoch 01831: loss did not improve from 1.65897
Epoch 1832/2000
240/240 [==============================] - 0s 647us/step - loss: 1.6591 - accuracy: 0.4042

Epoch 01832: loss did not improve from 1.65897
Epoch 1833/2000
240/240 [==============================] - 0s 628us/step - loss: 1.6590 - accuracy: 0.4125

Epoch 01833: loss did not improve from 1.65897
Epoch 1834/2000
240/240 [==============================] - 0s 630us/step - loss: 1.6590 - accuracy: 0.4208

Epoch 01834: loss did not improve from 1.65897
Epoch 1835/2000
240/240 [==============================] - 0s 625us/step - loss: 1.658

240/240 [==============================] - 0s 632us/step - loss: 1.6592 - accuracy: 0.4167

Epoch 01933: loss did not improve from 1.65887
Epoch 1934/2000
240/240 [==============================] - 0s 613us/step - loss: 1.6597 - accuracy: 0.4167

Epoch 01934: loss did not improve from 1.65887
Epoch 1935/2000
240/240 [==============================] - 0s 674us/step - loss: 1.6611 - accuracy: 0.4167

Epoch 01935: loss did not improve from 1.65887
Epoch 1936/2000
240/240 [==============================] - 0s 622us/step - loss: 1.6638 - accuracy: 0.4292

Epoch 01936: loss did not improve from 1.65887
Epoch 1937/2000
240/240 [==============================] - 0s 629us/step - loss: 1.6604 - accuracy: 0.4167

Epoch 01937: loss did not improve from 1.65887
Epoch 1938/2000
240/240 [==============================] - 0s 684us/step - loss: 1.6599 - accuracy: 0.4125

Epoch 01938: loss did not improve from 1.65887
Epoch 1939/2000
240/240 [==============================] - 0s 665us/step - loss: 1.659

In [9]:
readdata_and_savemodel("1000001000100010002.csv")

Epoch 1/2000
93/93 [==============================] - 0s 1ms/step - loss: 4.6620 - accuracy: 0.0000e+00

Epoch 00001: loss improved from inf to 4.66197, saving model to ./model/1000001000100010002.hdf5
Epoch 2/2000
93/93 [==============================] - 0s 754us/step - loss: 4.3926 - accuracy: 0.0430

Epoch 00002: loss improved from 4.66197 to 4.39259, saving model to ./model/1000001000100010002.hdf5
Epoch 3/2000
93/93 [==============================] - 0s 751us/step - loss: 4.0592 - accuracy: 0.1075

Epoch 00003: loss improved from 4.39259 to 4.05924, saving model to ./model/1000001000100010002.hdf5
Epoch 4/2000
93/93 [==============================] - 0s 718us/step - loss: 3.5953 - accuracy: 0.2043

Epoch 00004: loss improved from 4.05924 to 3.59526, saving model to ./model/1000001000100010002.hdf5
Epoch 5/2000
93/93 [==============================] - 0s 807us/step - loss: 3.1324 - accuracy: 0.2796

Epoch 00005: loss improved from 3.59526 to 3.13244, saving model to ./model/1000001

93/93 [==============================] - 0s 732us/step - loss: 1.7644 - accuracy: 0.4516

Epoch 00043: loss did not improve from 1.76008
Epoch 44/2000
93/93 [==============================] - 0s 758us/step - loss: 1.7576 - accuracy: 0.4516

Epoch 00044: loss improved from 1.76008 to 1.75765, saving model to ./model/1000001000100010002.hdf5
Epoch 45/2000
93/93 [==============================] - 0s 710us/step - loss: 1.7621 - accuracy: 0.4516

Epoch 00045: loss did not improve from 1.75765
Epoch 46/2000
93/93 [==============================] - 0s 729us/step - loss: 1.7665 - accuracy: 0.4516

Epoch 00046: loss did not improve from 1.75765
Epoch 47/2000
93/93 [==============================] - 0s 712us/step - loss: 1.7563 - accuracy: 0.4624

Epoch 00047: loss improved from 1.75765 to 1.75628, saving model to ./model/1000001000100010002.hdf5
Epoch 48/2000
93/93 [==============================] - 0s 718us/step - loss: 1.7522 - accuracy: 0.4624

Epoch 00048: loss improved from 1.75628 to 1.75

93/93 [==============================] - 0s 754us/step - loss: 1.7053 - accuracy: 0.4624

Epoch 00137: loss did not improve from 1.69603
Epoch 138/2000
93/93 [==============================] - 0s 760us/step - loss: 1.6984 - accuracy: 0.4624

Epoch 00138: loss did not improve from 1.69603
Epoch 139/2000
93/93 [==============================] - 0s 718us/step - loss: 1.6938 - accuracy: 0.4516

Epoch 00139: loss improved from 1.69603 to 1.69375, saving model to ./model/1000001000100010002.hdf5
Epoch 140/2000
93/93 [==============================] - 0s 722us/step - loss: 1.6936 - accuracy: 0.4839

Epoch 00140: loss improved from 1.69375 to 1.69362, saving model to ./model/1000001000100010002.hdf5
Epoch 141/2000
93/93 [==============================] - 0s 722us/step - loss: 1.6937 - accuracy: 0.4516

Epoch 00141: loss did not improve from 1.69362
Epoch 142/2000
93/93 [==============================] - 0s 715us/step - loss: 1.6938 - accuracy: 0.4624

Epoch 00142: loss did not improve from 1.6

93/93 [==============================] - 0s 831us/step - loss: 1.6849 - accuracy: 0.4516

Epoch 00236: loss did not improve from 1.68415
Epoch 237/2000
93/93 [==============================] - 0s 783us/step - loss: 1.6852 - accuracy: 0.4516

Epoch 00237: loss did not improve from 1.68415
Epoch 238/2000
93/93 [==============================] - 0s 772us/step - loss: 1.6843 - accuracy: 0.4624

Epoch 00238: loss did not improve from 1.68415
Epoch 239/2000
93/93 [==============================] - 0s 755us/step - loss: 1.6843 - accuracy: 0.4731

Epoch 00239: loss did not improve from 1.68415
Epoch 240/2000
93/93 [==============================] - 0s 722us/step - loss: 1.6846 - accuracy: 0.4624

Epoch 00240: loss did not improve from 1.68415
Epoch 241/2000
93/93 [==============================] - 0s 729us/step - loss: 1.6841 - accuracy: 0.4624

Epoch 00241: loss improved from 1.68415 to 1.68414, saving model to ./model/1000001000100010002.hdf5
Epoch 242/2000
93/93 [===========================

93/93 [==============================] - 0s 645us/step - loss: 1.6815 - accuracy: 0.4839

Epoch 00335: loss did not improve from 1.68126
Epoch 336/2000
93/93 [==============================] - 0s 611us/step - loss: 1.6814 - accuracy: 0.4516

Epoch 00336: loss did not improve from 1.68126
Epoch 337/2000
93/93 [==============================] - 0s 665us/step - loss: 1.6813 - accuracy: 0.4624

Epoch 00337: loss did not improve from 1.68126
Epoch 338/2000
93/93 [==============================] - 0s 654us/step - loss: 1.6813 - accuracy: 0.4624

Epoch 00338: loss did not improve from 1.68126
Epoch 339/2000
93/93 [==============================] - 0s 645us/step - loss: 1.6816 - accuracy: 0.4516

Epoch 00339: loss did not improve from 1.68126
Epoch 340/2000
93/93 [==============================] - 0s 643us/step - loss: 1.6814 - accuracy: 0.4516

Epoch 00340: loss did not improve from 1.68126
Epoch 341/2000
93/93 [==============================] - 0s 633us/step - loss: 1.6811 - accuracy: 0.4624

...

93/93 [==============================] - 0s 611us/step - loss: 1.6774 - accuracy: 0.4624

Epoch 01553: loss did not improve from 1.67561
Epoch 1554/2000
93/93 [==============================] - 0s 633us/step - loss: 1.6774 - accuracy: 0.4624

Epoch 01554: loss did not improve from 1.67561
Epoch 1555/2000
93/93 [==============================] - 0s 601us/step - loss: 1.6776 - accuracy: 0.4624

Epoch 01555: loss did not improve from 1.67561
Epoch 1556/2000
93/93 [==============================] - 0s 611us/step - loss: 1.6773 - accuracy: 0.4516

Epoch 01556: loss did not improve from 1.67561
Epoch 1557/2000
93/93 [==============================] - 0s 611us/step - loss: 1.6774 - accuracy: 0.4624

Epoch 01557: loss did not improve from 1.67561
Epoch 1558/2000
93/93 [==============================] - 0s 611us/step - loss: 1.6775 - accuracy: 0.4624

Epoch 01558: loss did not improve from 1.67561
Epoch 1559/2000
93/93 [==============================] - 0s 579us/step - loss: 1.6774 - accuracy: 

93/93 [==============================] - 0s 633us/step - loss: 1.6776 - accuracy: 0.4624

Epoch 01659: loss did not improve from 1.67561
Epoch 1660/2000
93/93 [==============================] - 0s 622us/step - loss: 1.6775 - accuracy: 0.4516

Epoch 01660: loss did not improve from 1.67561
Epoch 1661/2000
93/93 [==============================] - 0s 622us/step - loss: 1.6774 - accuracy: 0.4624

Epoch 01661: loss did not improve from 1.67561
Epoch 1662/2000
93/93 [==============================] - 0s 611us/step - loss: 1.6774 - accuracy: 0.4516

Epoch 01662: loss did not improve from 1.67561
Epoch 1663/2000
93/93 [==============================] - 0s 643us/step - loss: 1.6775 - accuracy: 0.4624

Epoch 01663: loss did not improve from 1.67561
Epoch 1664/2000
93/93 [==============================] - 0s 601us/step - loss: 1.6773 - accuracy: 0.4516

Epoch 01664: loss did not improve from 1.67561
Epoch 1665/2000
93/93 [==============================] - 0s 633us/step - loss: 1.6774 - accuracy: 

93/93 [==============================] - 0s 664us/step - loss: 1.6775 - accuracy: 0.4516

Epoch 01765: loss did not improve from 1.67561
Epoch 1766/2000
93/93 [==============================] - 0s 622us/step - loss: 1.6773 - accuracy: 0.4516

Epoch 01766: loss did not improve from 1.67561
Epoch 1767/2000
93/93 [==============================] - 0s 643us/step - loss: 1.6773 - accuracy: 0.4731

Epoch 01767: loss did not improve from 1.67561
Epoch 1768/2000
93/93 [==============================] - 0s 664us/step - loss: 1.6773 - accuracy: 0.4516

Epoch 01768: loss did not improve from 1.67561
Epoch 1769/2000
93/93 [==============================] - 0s 601us/step - loss: 1.6772 - accuracy: 0.4516

Epoch 01769: loss did not improve from 1.67561
Epoch 1770/2000
93/93 [==============================] - 0s 697us/step - loss: 1.6774 - accuracy: 0.4731

Epoch 01770: loss did not improve from 1.67561
Epoch 1771/2000
93/93 [==============================] - 0s 665us/step - loss: 1.6773 - accuracy: 

93/93 [==============================] - 0s 622us/step - loss: 1.6773 - accuracy: 0.4516

Epoch 01871: loss did not improve from 1.67561
Epoch 1872/2000
93/93 [==============================] - 0s 611us/step - loss: 1.6773 - accuracy: 0.4624

Epoch 01872: loss did not improve from 1.67561
Epoch 1873/2000
93/93 [==============================] - 0s 611us/step - loss: 1.6774 - accuracy: 0.4516

Epoch 01873: loss did not improve from 1.67561
Epoch 1874/2000
93/93 [==============================] - 0s 601us/step - loss: 1.6773 - accuracy: 0.4516

Epoch 01874: loss did not improve from 1.67561
Epoch 1875/2000
93/93 [==============================] - 0s 611us/step - loss: 1.6773 - accuracy: 0.4516

Epoch 01875: loss did not improve from 1.67561
Epoch 1876/2000
93/93 [==============================] - 0s 589us/step - loss: 1.6772 - accuracy: 0.4624

Epoch 01876: loss did not improve from 1.67561
Epoch 1877/2000
93/93 [==============================] - 0s 586us/step - loss: 1.6773 - accuracy: 

93/93 [==============================] - 0s 611us/step - loss: 1.6772 - accuracy: 0.4731

Epoch 01977: loss did not improve from 1.67561
Epoch 1978/2000
93/93 [==============================] - 0s 602us/step - loss: 1.6772 - accuracy: 0.4624

Epoch 01978: loss did not improve from 1.67561
Epoch 1979/2000
93/93 [==============================] - 0s 578us/step - loss: 1.6773 - accuracy: 0.4624

Epoch 01979: loss did not improve from 1.67561
Epoch 1980/2000
93/93 [==============================] - 0s 579us/step - loss: 1.6772 - accuracy: 0.4731

Epoch 01980: loss did not improve from 1.67561
Epoch 1981/2000
93/93 [==============================] - 0s 611us/step - loss: 1.6772 - accuracy: 0.4624

Epoch 01981: loss did not improve from 1.67561
Epoch 1982/2000
93/93 [==============================] - 0s 582us/step - loss: 1.6770 - accuracy: 0.4624

Epoch 01982: loss did not improve from 1.67561
Epoch 1983/2000
93/93 [==============================] - 0s 622us/step - loss: 1.6772 - accuracy: 

In [10]:
readdata_and_savemodel("1000001000100010003.csv")

Epoch 1/2000
240/240 [==============================] - 0s 1ms/step - loss: 5.5841 - accuracy: 0.0000e+00

Epoch 00001: loss improved from inf to 5.58407, saving model to ./model/1000001000100010003.hdf5
Epoch 2/2000
240/240 [==============================] - 0s 727us/step - loss: 5.3354 - accuracy: 0.0167

Epoch 00002: loss improved from 5.58407 to 5.33543, saving model to ./model/1000001000100010003.hdf5
Epoch 3/2000
240/240 [==============================] - 0s 749us/step - loss: 4.5788 - accuracy: 0.0917

Epoch 00003: loss improved from 5.33543 to 4.57881, saving model to ./model/1000001000100010003.hdf5
Epoch 4/2000
240/240 [==============================] - 0s 881us/step - loss: 3.5781 - accuracy: 0.2333

Epoch 00004: loss improved from 4.57881 to 3.57810, saving model to ./model/1000001000100010003.hdf5
Epoch 5/2000
240/240 [==============================] - 0s 823us/step - loss: 2.9487 - accuracy: 0.3083

Epoch 00005: loss improved from 3.57810 to 2.94869, saving model to ./mod

Epoch 43/2000
240/240 [==============================] - 0s 717us/step - loss: 2.1329 - accuracy: 0.3083

Epoch 00043: loss improved from 2.13599 to 2.13292, saving model to ./model/1000001000100010003.hdf5
Epoch 44/2000
240/240 [==============================] - 0s 723us/step - loss: 2.1394 - accuracy: 0.3333

Epoch 00044: loss did not improve from 2.13292
Epoch 45/2000
240/240 [==============================] - 0s 740us/step - loss: 2.1318 - accuracy: 0.3292

Epoch 00045: loss improved from 2.13292 to 2.13185, saving model to ./model/1000001000100010003.hdf5
Epoch 46/2000
240/240 [==============================] - 0s 728us/step - loss: 2.1277 - accuracy: 0.3292

Epoch 00046: loss improved from 2.13185 to 2.12773, saving model to ./model/1000001000100010003.hdf5
Epoch 47/2000
240/240 [==============================] - 0s 723us/step - loss: 2.1272 - accuracy: 0.3125

Epoch 00047: loss improved from 2.12773 to 2.12722, saving model to ./model/1000001000100010003.hdf5
Epoch 48/2000
240/2

Epoch 88/2000
240/240 [==============================] - 0s 898us/step - loss: 2.0713 - accuracy: 0.3500

Epoch 00088: loss improved from 2.07353 to 2.07127, saving model to ./model/1000001000100010003.hdf5
Epoch 89/2000
240/240 [==============================] - 0s 874us/step - loss: 2.0682 - accuracy: 0.3375

Epoch 00089: loss improved from 2.07127 to 2.06815, saving model to ./model/1000001000100010003.hdf5
Epoch 90/2000
240/240 [==============================] - 0s 799us/step - loss: 2.0700 - accuracy: 0.3292

Epoch 00090: loss did not improve from 2.06815
Epoch 91/2000
240/240 [==============================] - 0s 729us/step - loss: 2.0685 - accuracy: 0.3375

Epoch 00091: loss did not improve from 2.06815
Epoch 92/2000
240/240 [==============================] - 0s 675us/step - loss: 2.0673 - accuracy: 0.3167

Epoch 00092: loss improved from 2.06815 to 2.06728, saving model to ./model/1000001000100010003.hdf5
Epoch 93/2000
240/240 [==============================] - 0s 708us/step - 

240/240 [==============================] - 0s 631us/step - loss: 2.0387 - accuracy: 0.3333

Epoch 00181: loss did not improve from 2.03824
Epoch 182/2000
240/240 [==============================] - 0s 617us/step - loss: 2.0376 - accuracy: 0.3542

Epoch 00182: loss improved from 2.03824 to 2.03756, saving model to ./model/1000001000100010003.hdf5
Epoch 183/2000
240/240 [==============================] - 0s 706us/step - loss: 2.0385 - accuracy: 0.3500

Epoch 00183: loss did not improve from 2.03756
Epoch 184/2000
240/240 [==============================] - 0s 704us/step - loss: 2.0393 - accuracy: 0.3292

Epoch 00184: loss did not improve from 2.03756
Epoch 185/2000
240/240 [==============================] - 0s 702us/step - loss: 2.0377 - accuracy: 0.3292

Epoch 00185: loss did not improve from 2.03756
Epoch 186/2000
240/240 [==============================] - 0s 699us/step - loss: 2.0370 - accuracy: 0.3375

Epoch 00186: loss improved from 2.03756 to 2.03696, saving model to ./model/10000010

Epoch 230/2000
240/240 [==============================] - 0s 733us/step - loss: 2.0328 - accuracy: 0.3417

Epoch 00230: loss did not improve from 2.03151
Epoch 231/2000
240/240 [==============================] - 0s 708us/step - loss: 2.0299 - accuracy: 0.3208

Epoch 00231: loss improved from 2.03151 to 2.02993, saving model to ./model/1000001000100010003.hdf5
Epoch 232/2000
240/240 [==============================] - 0s 706us/step - loss: 2.0311 - accuracy: 0.3333

Epoch 00232: loss did not improve from 2.02993
Epoch 233/2000
240/240 [==============================] - 0s 716us/step - loss: 2.0317 - accuracy: 0.3542

Epoch 00233: loss did not improve from 2.02993
Epoch 234/2000
240/240 [==============================] - 0s 704us/step - loss: 2.0313 - accuracy: 0.3458

Epoch 00234: loss did not improve from 2.02993
Epoch 235/2000
240/240 [==============================] - 0s 701us/step - loss: 2.0323 - accuracy: 0.3417

Epoch 00235: loss did not improve from 2.02993
Epoch 236/2000
240/240

...

240/240 [==============================] - 0s 688us/step - loss: 2.0155 - accuracy: 0.3500

Epoch 01575: loss did not improve from 2.01539
Epoch 1576/2000
240/240 [==============================] - 0s 787us/step - loss: 2.0154 - accuracy: 0.3208

Epoch 01576: loss did not improve from 2.01539
Epoch 1577/2000
240/240 [==============================] - 0s 786us/step - loss: 2.0157 - accuracy: 0.3333

Epoch 01577: loss did not improve from 2.01539
Epoch 1578/2000
240/240 [==============================] - 0s 744us/step - loss: 2.0159 - accuracy: 0.3292

Epoch 01578: loss did not improve from 2.01539
Epoch 1579/2000
240/240 [==============================] - 0s 777us/step - loss: 2.0158 - accuracy: 0.3542

Epoch 01579: loss did not improve from 2.01539
Epoch 1580/2000
240/240 [==============================] - 0s 747us/step - loss: 2.0158 - accuracy: 0.3417

Epoch 01580: loss did not improve from 2.01539
Epoch 1581/2000
240/240 [==============================] - 0s 777us/step - loss: 2.015

240/240 [==============================] - 0s 697us/step - loss: 2.0157 - accuracy: 0.3500

Epoch 01679: loss did not improve from 2.01523
Epoch 1680/2000
240/240 [==============================] - 0s 683us/step - loss: 2.0156 - accuracy: 0.3375

Epoch 01680: loss did not improve from 2.01523
Epoch 1681/2000
240/240 [==============================] - 0s 719us/step - loss: 2.0156 - accuracy: 0.3250

Epoch 01681: loss did not improve from 2.01523
Epoch 1682/2000
240/240 [==============================] - 0s 696us/step - loss: 2.0154 - accuracy: 0.3583

Epoch 01682: loss did not improve from 2.01523
Epoch 1683/2000
240/240 [==============================] - 0s 683us/step - loss: 2.0152 - accuracy: 0.3375

Epoch 01683: loss did not improve from 2.01523
Epoch 1684/2000
240/240 [==============================] - 0s 693us/step - loss: 2.0155 - accuracy: 0.3417

Epoch 01684: loss did not improve from 2.01523
Epoch 1685/2000
240/240 [==============================] - 0s 733us/step - loss: 2.015

240/240 [==============================] - 0s 694us/step - loss: 2.0154 - accuracy: 0.3500

Epoch 01783: loss did not improve from 2.01501
Epoch 1784/2000
240/240 [==============================] - 0s 689us/step - loss: 2.0154 - accuracy: 0.3333

Epoch 01784: loss did not improve from 2.01501
Epoch 1785/2000
240/240 [==============================] - 0s 682us/step - loss: 2.0162 - accuracy: 0.3458

Epoch 01785: loss did not improve from 2.01501
Epoch 1786/2000
240/240 [==============================] - 0s 680us/step - loss: 2.0172 - accuracy: 0.3375

Epoch 01786: loss did not improve from 2.01501
Epoch 1787/2000
240/240 [==============================] - 0s 670us/step - loss: 2.0158 - accuracy: 0.3333

Epoch 01787: loss did not improve from 2.01501
Epoch 1788/2000
240/240 [==============================] - 0s 673us/step - loss: 2.0158 - accuracy: 0.3208

Epoch 01788: loss did not improve from 2.01501
Epoch 1789/2000
240/240 [==============================] - 0s 691us/step - loss: 2.015

240/240 [==============================] - 0s 712us/step - loss: 2.0153 - accuracy: 0.3333

Epoch 01888: loss did not improve from 2.01492
Epoch 1889/2000
240/240 [==============================] - 0s 687us/step - loss: 2.0157 - accuracy: 0.3417

Epoch 01889: loss did not improve from 2.01492
Epoch 1890/2000
240/240 [==============================] - 0s 728us/step - loss: 2.0153 - accuracy: 0.3250

Epoch 01890: loss did not improve from 2.01492
Epoch 1891/2000
240/240 [==============================] - 0s 677us/step - loss: 2.0160 - accuracy: 0.3417

Epoch 01891: loss did not improve from 2.01492
Epoch 1892/2000
240/240 [==============================] - 0s 700us/step - loss: 2.0167 - accuracy: 0.3333

Epoch 01892: loss did not improve from 2.01492
Epoch 1893/2000
240/240 [==============================] - 0s 737us/step - loss: 2.0173 - accuracy: 0.3458

Epoch 01893: loss did not improve from 2.01492
Epoch 1894/2000
240/240 [==============================] - 0s 727us/step - loss: 2.015


Epoch 01940: loss did not improve from 2.01492
Epoch 1941/2000
240/240 [==============================] - 0s 707us/step - loss: 2.0165 - accuracy: 0.3250

Epoch 01941: loss did not improve from 2.01492
Epoch 1942/2000
240/240 [==============================] - 0s 684us/step - loss: 2.0161 - accuracy: 0.3417

Epoch 01942: loss did not improve from 2.01492
Epoch 1943/2000
240/240 [==============================] - 0s 670us/step - loss: 2.0157 - accuracy: 0.3458

Epoch 01943: loss did not improve from 2.01492
Epoch 1944/2000
240/240 [==============================] - 0s 674us/step - loss: 2.0155 - accuracy: 0.3250

Epoch 01944: loss did not improve from 2.01492
Epoch 1945/2000
240/240 [==============================] - 0s 804us/step - loss: 2.0155 - accuracy: 0.3250

Epoch 01945: loss did not improve from 2.01492
Epoch 1946/2000
240/240 [==============================] - 0s 880us/step - loss: 2.0154 - accuracy: 0.3250

Epoch 01946: loss did not improve from 2.01492
Epoch 1947/2000
240/24

In [11]:
readdata_and_savemodel("1000001000100010004.csv")

Epoch 1/2000
240/240 [==============================] - 0s 1ms/step - loss: 5.5959 - accuracy: 0.0000e+00

Epoch 00001: loss improved from inf to 5.59586, saving model to ./model/1000001000100010004.hdf5
Epoch 2/2000
240/240 [==============================] - 0s 831us/step - loss: 5.2250 - accuracy: 0.0292

Epoch 00002: loss improved from 5.59586 to 5.22500, saving model to ./model/1000001000100010004.hdf5
Epoch 3/2000
240/240 [==============================] - 0s 826us/step - loss: 4.1193 - accuracy: 0.1000

Epoch 00003: loss improved from 5.22500 to 4.11933, saving model to ./model/1000001000100010004.hdf5
Epoch 4/2000
240/240 [==============================] - 0s 808us/step - loss: 2.8861 - accuracy: 0.2542

Epoch 00004: loss improved from 4.11933 to 2.88613, saving model to ./model/1000001000100010004.hdf5
Epoch 5/2000
240/240 [==============================] - 0s 803us/step - loss: 2.2068 - accuracy: 0.3875

Epoch 00005: loss improved from 2.88613 to 2.20680, saving model to ./mod

240/240 [==============================] - 0s 703us/step - loss: 1.2957 - accuracy: 0.4250

Epoch 00085: loss improved from 1.29960 to 1.29572, saving model to ./model/1000001000100010004.hdf5
Epoch 86/2000
240/240 [==============================] - 0s 716us/step - loss: 1.2969 - accuracy: 0.4292

Epoch 00086: loss did not improve from 1.29572
Epoch 87/2000
240/240 [==============================] - 0s 710us/step - loss: 1.2922 - accuracy: 0.4417

Epoch 00087: loss improved from 1.29572 to 1.29221, saving model to ./model/1000001000100010004.hdf5
Epoch 88/2000
240/240 [==============================] - 0s 727us/step - loss: 1.2933 - accuracy: 0.4417

Epoch 00088: loss did not improve from 1.29221
Epoch 89/2000
240/240 [==============================] - 0s 704us/step - loss: 1.2921 - accuracy: 0.4375

Epoch 00089: loss improved from 1.29221 to 1.29212, saving model to ./model/1000001000100010004.hdf5
Epoch 90/2000
240/240 [==============================] - 0s 706us/step - loss: 1.2935 -

Epoch 131/2000
240/240 [==============================] - 0s 707us/step - loss: 1.2696 - accuracy: 0.4458

Epoch 00131: loss did not improve from 1.26948
Epoch 132/2000
240/240 [==============================] - 0s 715us/step - loss: 1.2723 - accuracy: 0.4375

Epoch 00132: loss did not improve from 1.26948
Epoch 133/2000
240/240 [==============================] - 0s 707us/step - loss: 1.2707 - accuracy: 0.4375

Epoch 00133: loss did not improve from 1.26948
Epoch 134/2000
240/240 [==============================] - 0s 747us/step - loss: 1.2713 - accuracy: 0.4125

Epoch 00134: loss did not improve from 1.26948
Epoch 135/2000
240/240 [==============================] - 0s 706us/step - loss: 1.2696 - accuracy: 0.4500

Epoch 00135: loss did not improve from 1.26948
Epoch 136/2000
240/240 [==============================] - 0s 710us/step - loss: 1.2699 - accuracy: 0.4542

Epoch 00136: loss did not improve from 1.26948
Epoch 137/2000
240/240 [==============================] - 0s 716us/step - lo

240/240 [==============================] - 0s 681us/step - loss: 1.2516 - accuracy: 0.4417

Epoch 00229: loss did not improve from 1.25153
Epoch 230/2000
240/240 [==============================] - 0s 698us/step - loss: 1.2550 - accuracy: 0.4292

Epoch 00230: loss did not improve from 1.25153
Epoch 231/2000
240/240 [==============================] - 0s 667us/step - loss: 1.2533 - accuracy: 0.4292

Epoch 00231: loss did not improve from 1.25153
Epoch 232/2000
240/240 [==============================] - 0s 623us/step - loss: 1.2575 - accuracy: 0.4542

Epoch 00232: loss did not improve from 1.25153
Epoch 233/2000
240/240 [==============================] - 0s 727us/step - loss: 1.2545 - accuracy: 0.4167

Epoch 00233: loss did not improve from 1.25153
Epoch 234/2000
240/240 [==============================] - 0s 748us/step - loss: 1.2533 - accuracy: 0.4333

Epoch 00234: loss did not improve from 1.25153
Epoch 235/2000
240/240 [==============================] - 0s 741us/step - loss: 1.2515 - ac

240/240 [==============================] - 0s 686us/step - loss: 1.2474 - accuracy: 0.4625

Epoch 00279: loss improved from 1.24789 to 1.24739, saving model to ./model/1000001000100010004.hdf5
Epoch 280/2000
240/240 [==============================] - 0s 682us/step - loss: 1.2483 - accuracy: 0.4458

Epoch 00280: loss did not improve from 1.24739
Epoch 281/2000
240/240 [==============================] - 0s 669us/step - loss: 1.2482 - accuracy: 0.4250

Epoch 00281: loss did not improve from 1.24739
Epoch 282/2000
240/240 [==============================] - 0s 631us/step - loss: 1.2468 - accuracy: 0.4333

Epoch 00282: loss improved from 1.24739 to 1.24684, saving model to ./model/1000001000100010004.hdf5
Epoch 283/2000
240/240 [==============================] - 0s 704us/step - loss: 1.2471 - accuracy: 0.4375

Epoch 00283: loss did not improve from 1.24684
Epoch 284/2000
240/240 [==============================] - 0s 702us/step - loss: 1.2475 - accuracy: 0.4500

Epoch 00284: loss did not impr

...

240/240 [==============================] - 0s 630us/step - loss: 1.2369 - accuracy: 0.4375

Epoch 01571: loss did not improve from 1.23554
Epoch 1572/2000
240/240 [==============================] - 0s 630us/step - loss: 1.2364 - accuracy: 0.4458

Epoch 01572: loss did not improve from 1.23554
Epoch 1573/2000
240/240 [==============================] - 0s 636us/step - loss: 1.2363 - accuracy: 0.4208

Epoch 01573: loss did not improve from 1.23554
Epoch 1574/2000
240/240 [==============================] - 0s 655us/step - loss: 1.2360 - accuracy: 0.4208

Epoch 01574: loss did not improve from 1.23554
Epoch 1575/2000
240/240 [==============================] - 0s 634us/step - loss: 1.2362 - accuracy: 0.4292

Epoch 01575: loss did not improve from 1.23554
Epoch 1576/2000
240/240 [==============================] - 0s 626us/step - loss: 1.2362 - accuracy: 0.4125

Epoch 01576: loss did not improve from 1.23554
Epoch 1577/2000
240/240 [==============================] - 0s 650us/step - loss: 1.236

240/240 [==============================] - 0s 678us/step - loss: 1.2385 - accuracy: 0.4417

Epoch 01675: loss did not improve from 1.23543
Epoch 1676/2000
240/240 [==============================] - 0s 674us/step - loss: 1.2369 - accuracy: 0.4250

Epoch 01676: loss did not improve from 1.23543
Epoch 1677/2000
240/240 [==============================] - 0s 679us/step - loss: 1.2364 - accuracy: 0.4417

Epoch 01677: loss did not improve from 1.23543
Epoch 1678/2000
240/240 [==============================] - 0s 670us/step - loss: 1.2364 - accuracy: 0.4417

Epoch 01678: loss did not improve from 1.23543
Epoch 1679/2000
240/240 [==============================] - 0s 669us/step - loss: 1.2363 - accuracy: 0.4375

Epoch 01679: loss did not improve from 1.23543
Epoch 1680/2000
240/240 [==============================] - 0s 694us/step - loss: 1.2360 - accuracy: 0.4417

Epoch 01680: loss did not improve from 1.23543
Epoch 1681/2000
240/240 [==============================] - 0s 767us/step - loss: 1.236

240/240 [==============================] - 0s 634us/step - loss: 1.2371 - accuracy: 0.4250

Epoch 01780: loss did not improve from 1.23527
Epoch 1781/2000
240/240 [==============================] - 0s 648us/step - loss: 1.2374 - accuracy: 0.4292

Epoch 01781: loss did not improve from 1.23527
Epoch 1782/2000
240/240 [==============================] - 0s 630us/step - loss: 1.2370 - accuracy: 0.4208

Epoch 01782: loss did not improve from 1.23527
Epoch 1783/2000
240/240 [==============================] - 0s 655us/step - loss: 1.2710 - accuracy: 0.4333

Epoch 01783: loss did not improve from 1.23527
Epoch 1784/2000
240/240 [==============================] - 0s 695us/step - loss: 1.2367 - accuracy: 0.4208

Epoch 01784: loss did not improve from 1.23527
Epoch 1785/2000
240/240 [==============================] - 0s 704us/step - loss: 1.2361 - accuracy: 0.4167

Epoch 01785: loss did not improve from 1.23527
Epoch 1786/2000
240/240 [==============================] - 0s 683us/step - loss: 1.236

240/240 [==============================] - 0s 645us/step - loss: 1.2354 - accuracy: 0.4417

Epoch 01833: loss did not improve from 1.23527
Epoch 1834/2000
240/240 [==============================] - 0s 639us/step - loss: 1.2355 - accuracy: 0.4375

Epoch 01834: loss did not improve from 1.23527
Epoch 1835/2000
240/240 [==============================] - 0s 691us/step - loss: 1.2356 - accuracy: 0.4583

Epoch 01835: loss did not improve from 1.23527
Epoch 1836/2000
240/240 [==============================] - 0s 644us/step - loss: 1.2354 - accuracy: 0.4375

Epoch 01836: loss did not improve from 1.23527
Epoch 1837/2000
240/240 [==============================] - 0s 663us/step - loss: 1.2355 - accuracy: 0.4458

Epoch 01837: loss did not improve from 1.23527
Epoch 1838/2000
240/240 [==============================] - 0s 629us/step - loss: 1.2357 - accuracy: 0.4417

Epoch 01838: loss did not improve from 1.23527
Epoch 1839/2000
240/240 [==============================] - 0s 621us/step - loss: 1.239

240/240 [==============================] - 0s 638us/step - loss: 1.2354 - accuracy: 0.4417

Epoch 01937: loss did not improve from 1.23505
Epoch 1938/2000
240/240 [==============================] - 0s 626us/step - loss: 1.2354 - accuracy: 0.4083

Epoch 01938: loss did not improve from 1.23505
Epoch 1939/2000
240/240 [==============================] - 0s 630us/step - loss: 1.2353 - accuracy: 0.4333

Epoch 01939: loss did not improve from 1.23505
Epoch 1940/2000
240/240 [==============================] - 0s 634us/step - loss: 1.2352 - accuracy: 0.4417

Epoch 01940: loss did not improve from 1.23505
Epoch 1941/2000
240/240 [==============================] - 0s 625us/step - loss: 1.2354 - accuracy: 0.4292

Epoch 01941: loss did not improve from 1.23505
Epoch 1942/2000
240/240 [==============================] - 0s 619us/step - loss: 1.2353 - accuracy: 0.4625

Epoch 01942: loss did not improve from 1.23505
Epoch 1943/2000
240/240 [==============================] - 0s 631us/step - loss: 1.235

In [12]:
readdata_and_savemodel("1000001000100010011.csv")

Epoch 1/2000
108/108 [==============================] - 0s 1ms/step - loss: 4.8216 - accuracy: 0.0000e+00

Epoch 00001: loss improved from inf to 4.82161, saving model to ./model/1000001000100010011.hdf5
Epoch 2/2000
108/108 [==============================] - 0s 923us/step - loss: 4.5535 - accuracy: 0.0556

Epoch 00002: loss improved from 4.82161 to 4.55354, saving model to ./model/1000001000100010011.hdf5
Epoch 3/2000
108/108 [==============================] - 0s 803us/step - loss: 4.1951 - accuracy: 0.0833

Epoch 00003: loss improved from 4.55354 to 4.19508, saving model to ./model/1000001000100010011.hdf5
Epoch 4/2000
108/108 [==============================] - 0s 877us/step - loss: 3.6757 - accuracy: 0.1204

Epoch 00004: loss improved from 4.19508 to 3.67569, saving model to ./model/1000001000100010011.hdf5
Epoch 5/2000
108/108 [==============================] - 0s 803us/step - loss: 3.1438 - accuracy: 0.2500

Epoch 00005: loss improved from 3.67569 to 3.14384, saving model to ./mod


Epoch 00043: loss improved from 1.67867 to 1.67515, saving model to ./model/1000001000100010011.hdf5
Epoch 44/2000
108/108 [==============================] - 0s 712us/step - loss: 1.6760 - accuracy: 0.4074

Epoch 00044: loss did not improve from 1.67515
Epoch 45/2000
108/108 [==============================] - 0s 680us/step - loss: 1.6758 - accuracy: 0.4167

Epoch 00045: loss did not improve from 1.67515
Epoch 46/2000
108/108 [==============================] - 0s 689us/step - loss: 1.6655 - accuracy: 0.3981

Epoch 00046: loss improved from 1.67515 to 1.66551, saving model to ./model/1000001000100010011.hdf5
Epoch 47/2000
108/108 [==============================] - 0s 726us/step - loss: 1.6656 - accuracy: 0.3981

Epoch 00047: loss did not improve from 1.66551
Epoch 48/2000
108/108 [==============================] - 0s 729us/step - loss: 1.6628 - accuracy: 0.3981

Epoch 00048: loss improved from 1.66551 to 1.66282, saving model to ./model/1000001000100010011.hdf5
Epoch 49/2000
108/108 [==

108/108 [==============================] - 0s 725us/step - loss: 1.5909 - accuracy: 0.3981

Epoch 00137: loss did not improve from 1.59072
Epoch 138/2000
108/108 [==============================] - 0s 735us/step - loss: 1.5920 - accuracy: 0.4074

Epoch 00138: loss did not improve from 1.59072
Epoch 139/2000
108/108 [==============================] - 0s 731us/step - loss: 1.5925 - accuracy: 0.3981

Epoch 00139: loss did not improve from 1.59072
Epoch 140/2000
108/108 [==============================] - 0s 725us/step - loss: 1.5923 - accuracy: 0.4074

Epoch 00140: loss did not improve from 1.59072
Epoch 141/2000
108/108 [==============================] - 0s 735us/step - loss: 1.5919 - accuracy: 0.3889

Epoch 00141: loss did not improve from 1.59072
Epoch 142/2000
108/108 [==============================] - 0s 738us/step - loss: 1.5919 - accuracy: 0.3981

Epoch 00142: loss did not improve from 1.59072
Epoch 143/2000
108/108 [==============================] - 0s 754us/step - loss: 1.5967 - ac

108/108 [==============================] - 0s 750us/step - loss: 1.5762 - accuracy: 0.3981

Epoch 00234: loss improved from 1.57638 to 1.57616, saving model to ./model/1000001000100010011.hdf5
Epoch 235/2000
108/108 [==============================] - 0s 734us/step - loss: 1.5767 - accuracy: 0.4167

Epoch 00235: loss did not improve from 1.57616
Epoch 236/2000
108/108 [==============================] - 0s 734us/step - loss: 1.5773 - accuracy: 0.3889

Epoch 00236: loss did not improve from 1.57616
Epoch 237/2000
108/108 [==============================] - 0s 739us/step - loss: 1.5769 - accuracy: 0.4074

Epoch 00237: loss did not improve from 1.57616
Epoch 238/2000
108/108 [==============================] - 0s 744us/step - loss: 1.5773 - accuracy: 0.3981

Epoch 00238: loss did not improve from 1.57616
Epoch 239/2000
108/108 [==============================] - 0s 731us/step - loss: 1.5759 - accuracy: 0.4074

Epoch 00239: loss improved from 1.57616 to 1.57593, saving model to ./model/10000010

108/108 [==============================] - 0s 745us/step - loss: 1.5708 - accuracy: 0.3889

Epoch 00333: loss did not improve from 1.57080
Epoch 334/2000
108/108 [==============================] - 0s 739us/step - loss: 1.5728 - accuracy: 0.4074

Epoch 00334: loss did not improve from 1.57080
Epoch 335/2000
108/108 [==============================] - 0s 742us/step - loss: 1.5707 - accuracy: 0.4167

Epoch 00335: loss improved from 1.57080 to 1.57072, saving model to ./model/1000001000100010011.hdf5
Epoch 336/2000
108/108 [==============================] - 0s 755us/step - loss: 1.5717 - accuracy: 0.3981

Epoch 00336: loss did not improve from 1.57072
Epoch 337/2000
108/108 [==============================] - 0s 722us/step - loss: 1.5702 - accuracy: 0.4167

Epoch 00337: loss improved from 1.57072 to 1.57023, saving model to ./model/1000001000100010011.hdf5
Epoch 338/2000
108/108 [==============================] - 0s 730us/step - loss: 1.5703 - accuracy: 0.4074

Epoch 00338: loss did not impr

...

108/108 [==============================] - 0s 619us/step - loss: 1.5621 - accuracy: 0.3981

Epoch 01583: loss did not improve from 1.56185
Epoch 1584/2000
108/108 [==============================] - 0s 609us/step - loss: 1.5621 - accuracy: 0.4074

Epoch 01584: loss did not improve from 1.56185
Epoch 1585/2000
108/108 [==============================] - 0s 598us/step - loss: 1.5622 - accuracy: 0.4074

Epoch 01585: loss did not improve from 1.56185
Epoch 1586/2000
108/108 [==============================] - 0s 600us/step - loss: 1.5623 - accuracy: 0.4259

Epoch 01586: loss did not improve from 1.56185
Epoch 1587/2000
108/108 [==============================] - 0s 591us/step - loss: 1.5621 - accuracy: 0.4259

Epoch 01587: loss did not improve from 1.56185
Epoch 1588/2000
108/108 [==============================] - 0s 594us/step - loss: 1.5624 - accuracy: 0.4167

Epoch 01588: loss did not improve from 1.56185
Epoch 1589/2000
108/108 [==============================] - 0s 587us/step - loss: 1.562

108/108 [==============================] - 0s 600us/step - loss: 1.5623 - accuracy: 0.3981

Epoch 01636: loss did not improve from 1.56185
Epoch 1637/2000
108/108 [==============================] - 0s 591us/step - loss: 1.5624 - accuracy: 0.3889

Epoch 01637: loss did not improve from 1.56185
Epoch 1638/2000
108/108 [==============================] - 0s 609us/step - loss: 1.5621 - accuracy: 0.3981

Epoch 01638: loss did not improve from 1.56185
Epoch 1639/2000
108/108 [==============================] - 0s 646us/step - loss: 1.5623 - accuracy: 0.4074

Epoch 01639: loss did not improve from 1.56185
Epoch 1640/2000
108/108 [==============================] - 0s 589us/step - loss: 1.5627 - accuracy: 0.4074

Epoch 01640: loss did not improve from 1.56185
Epoch 1641/2000
108/108 [==============================] - 0s 591us/step - loss: 1.5626 - accuracy: 0.4167

Epoch 01641: loss did not improve from 1.56185
Epoch 1642/2000
108/108 [==============================] - 0s 600us/step - loss: 1.562

108/108 [==============================] - 0s 608us/step - loss: 1.5619 - accuracy: 0.3889

Epoch 01741: loss did not improve from 1.56177
Epoch 1742/2000
108/108 [==============================] - 0s 609us/step - loss: 1.5621 - accuracy: 0.4259

Epoch 01742: loss did not improve from 1.56177
Epoch 1743/2000
108/108 [==============================] - 0s 599us/step - loss: 1.5621 - accuracy: 0.4167

Epoch 01743: loss did not improve from 1.56177
Epoch 1744/2000
108/108 [==============================] - 0s 619us/step - loss: 1.5621 - accuracy: 0.4074

Epoch 01744: loss did not improve from 1.56177
Epoch 1745/2000
108/108 [==============================] - 0s 618us/step - loss: 1.5621 - accuracy: 0.3889

Epoch 01745: loss did not improve from 1.56177
Epoch 1746/2000
108/108 [==============================] - 0s 600us/step - loss: 1.5621 - accuracy: 0.3889

Epoch 01746: loss did not improve from 1.56177
Epoch 1747/2000
108/108 [==============================] - 0s 591us/step - loss: 1.562

108/108 [==============================] - 0s 665us/step - loss: 1.5617 - accuracy: 0.3796

Epoch 01845: loss did not improve from 1.56163
Epoch 1846/2000
108/108 [==============================] - 0s 643us/step - loss: 1.5617 - accuracy: 0.4074

Epoch 01846: loss did not improve from 1.56163
Epoch 1847/2000
108/108 [==============================] - 0s 646us/step - loss: 1.5620 - accuracy: 0.4074

Epoch 01847: loss did not improve from 1.56163
Epoch 1848/2000
108/108 [==============================] - 0s 711us/step - loss: 1.5618 - accuracy: 0.4074

Epoch 01848: loss did not improve from 1.56163
Epoch 1849/2000
108/108 [==============================] - 0s 642us/step - loss: 1.5619 - accuracy: 0.3889

Epoch 01849: loss did not improve from 1.56163
Epoch 1850/2000
108/108 [==============================] - 0s 674us/step - loss: 1.5620 - accuracy: 0.4074

Epoch 01850: loss did not improve from 1.56163
Epoch 1851/2000
108/108 [==============================] - 0s 651us/step - loss: 1.561

108/108 [==============================] - 0s 642us/step - loss: 1.5619 - accuracy: 0.3981

Epoch 01898: loss did not improve from 1.56163
Epoch 1899/2000
108/108 [==============================] - 0s 636us/step - loss: 1.5619 - accuracy: 0.3796

Epoch 01899: loss did not improve from 1.56163
Epoch 1900/2000
108/108 [==============================] - 0s 637us/step - loss: 1.5625 - accuracy: 0.3889

Epoch 01900: loss did not improve from 1.56163
Epoch 1901/2000
108/108 [==============================] - 0s 636us/step - loss: 1.5630 - accuracy: 0.4167

Epoch 01901: loss did not improve from 1.56163
Epoch 1902/2000
108/108 [==============================] - 0s 653us/step - loss: 1.5636 - accuracy: 0.3981

Epoch 01902: loss did not improve from 1.56163
Epoch 1903/2000
108/108 [==============================] - 0s 636us/step - loss: 1.5620 - accuracy: 0.3796

Epoch 01903: loss did not improve from 1.56163
Epoch 1904/2000
108/108 [==============================] - 0s 581us/step - loss: 1.562

### 사용 예시

In [6]:
a,b,c = predict_code_value("크림",["복합성",'쿨톤'])
print(a,b,c)

FileNotFoundError: [Errno 2] No such file or directory: '1000001000100010001.csv'

# 간이 프로그래밍

In [7]:
print(category_tag_to_dictionary("크림",["지성",'웜톤','트러블','미백']))
    

FileNotFoundError: [Errno 2] No such file or directory: '1000001000100010001.csv'

In [101]:
def run():
    Flag = True
    name_list = ["복합성","건성","지성","쿨톤","웜톤","잡티","미백","주름","각질","트러블","블랙헤드","피지과다","민감성","모공","탄력","홍조","아토피"]
    while Flag:
        category = input("제품 카테고리를 입력해 주세요(스킨/토너, 크림, 로션, 에센스/세럼, 앰플) : ")
        print("\n {}을 선택 해 주셨네요. ".format(category))
        type_ = input("피부 타입을 입력해 주세요. (지성,건성,복합성)")
        print("\n {}을 선택 해 주셨네요. ".format(type_))
        ton = input("피부 톤을 입력해 주세요 (웜톤,쿨톤)")
        print("\n {} 을 선택해 주셨네요. ".format(ton))
#         print("이제 당신의 피부 고민을 선택해 주세요.")
        trouble = [type_,ton]
        while True :
            print("당신의 피부 고민을 선택해 주세요. (없을 0 입력)")
            print(name_list[5:])
            print("선택 listt = ",trouble)
            string = input()
            if string=='0':
                break
            else:
                trouble.append(string)
        a,b,c = predict_code_value(category,trouble)
        print("제품명 : ",a)
        print("가격 : ",b)
        print("link : ",c)
        reply = input("한번더 추천해 드릴까요? choice y/n")
        if reply == 'n':
            break
    return

In [ ]:
run()

제품 카테고리를 입력해 주세요(스킨/토너, 크림, 로션, 에센스/세럼, 앰플) : 스킨

 스킨을 선택 해 주셨네요. 
피부 타입을 입력해 주세요. (지성,건성,복합성)복합성

 복합성을 선택 해 주셨네요. 
피부 톤을 입력해 주세요 (웜톤,쿨톤)웜톤

 웜톤 을 선택해 주셨네요. 
당신의 피부 고민을 선택해 주세요. (없을 0 입력)
['잡티', '미백', '주름', '각질', '트러블', '블랙헤드', '피지과다', '민감성', '모공', '탄력', '홍조', '아토피']
선택 listt =  ['복합성', '웜톤']
트러블
당신의 피부 고민을 선택해 주세요. (없을 0 입력)
['잡티', '미백', '주름', '각질', '트러블', '블랙헤드', '피지과다', '민감성', '모공', '탄력', '홍조', '아토피']
선택 listt =  ['복합성', '웜톤', '트러블']
0
제품명 :  닥터지로얄블랙스네일퍼스트에센스165mL
가격 :  21,420
link :  https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000142567&dispCatNo=1000001000100010001
한번더 추천해 드릴까요? choice y/ny
제품 카테고리를 입력해 주세요(스킨/토너, 크림, 로션, 에센스/세럼, 앰플) : 로션

 로션을 선택 해 주셨네요. 
피부 타입을 입력해 주세요. (지성,건성,복합성)지성

 지성을 선택 해 주셨네요. 
피부 톤을 입력해 주세요 (웜톤,쿨톤)웜톤

 웜톤 을 선택해 주셨네요. 
당신의 피부 고민을 선택해 주세요. (없을 0 입력)
['잡티', '미백', '주름', '각질', '트러블', '블랙헤드', '피지과다', '민감성', '모공', '탄력', '홍조', '아토피']
선택 listt =  ['지성', '웜톤']
홍조
당신의 피부 고민을 선택해 주세요. (없을 0 입력)
['잡티', '미백', '주름', '각질', '트러블', '블랙헤드', '피지과